# Exercise 7: Advantage Actor-Critic (A2C)

In [ ]:
import numpy as np
import os
import tensorflow as tf

from unityagents import UnityEnvironment

### Hyperparameters

In [ ]:
y = 0.99 # Discount rate.
total_episodes = 20000 # Set total number of episodes to train agent on.
update_frequency = 20 # How many episodes before updating model.
summary_freq = 100 # How often to display information about training
learning_rate = 1e-3 # Agent learning rate.
hidden_units = 32 # Number of units in hidden layer.
model_path = "./models/ac" # The path to save our model to.
summary_path = "./summaries/ac" # The path to save our model to.
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.

### Load the environment

In [ ]:
env = UnityEnvironment("./envs/2DBall", worker_id=6)
default_brain = env.brain_names[0]

### Examine state space

In [ ]:
brains = env.reset()
agent_brain = brains[default_brain]
print(agent_brain.vector_observations)

State *(s)* is a vector whose values corresponds to the rotation of the platform and the position and velocity of the ball.

### The Actor Critic Agent

In [ ]:
class ACAgent():
    def __init__(self, lr, s_size, a_size, h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None, s_size],dtype=tf.float32)
        hidden = tf.layers.dense(self.state_in, h_size, activation=tf.nn.elu)
        self.policy = tf.layers.dense(hidden, a_size, activation=tf.nn.softmax)
        self.sampled_action = tf.multinomial(self.policy, 1)
        self.value = tf.layers.dense(hidden, 1, activation=None)

        # The randomness in the action probabilities.
        self.entropy = -tf.reduce_sum(self.policy * tf.log(self.policy + 1e-10), axis=1)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions = tf.one_hot(self.action_holder, a_size)
        
        self.taken_action = tf.reduce_sum(self.policy * self.actions, axis=1)
        
        advantage = self.reward_holder - tf.stop_gradient(tf.reduce_sum(self.value, axis=1))
        mean_adv, var_adv = tf.nn.moments(advantage, axes=0)
        normalized_advantage = (advantage - mean_adv) / (tf.sqrt(var_adv) + 1e-10)
        
        self.policy_loss = -tf.reduce_mean(tf.log(self.taken_action + 1e-10) * normalized_advantage)
        self.value_loss = tf.reduce_mean(tf.squared_difference(self.reward_holder,tf.reduce_sum(self.value,axis=1)))
        self.loss = self.policy_loss + 0.5 * self.value_loss - 0.01 * tf.reduce_mean(self.entropy)
        
        # Here we apply the gradients to update the network. 
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_model = optimizer.minimize(self.loss)

### Training the Agent

In [ ]:
# Function to take list of rewards and discount factor
# and produce discounted sum of future rewards.
def discount_rewards(r, gamma):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [ ]:
if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(summary_path):
    os.makedirs(summary_path)

#Load the agent.
tf.reset_default_graph() 
state_size = env.brains[default_brain].vector_observation_space_size * env.brains[default_brain].num_stacked_vector_observations
action_size = env.brains[default_brain].vector_action_space_size
myAgent = ACAgent(lr=learning_rate, s_size=state_size, 
                  a_size=action_size, h_size=hidden_units)

# Start an interactive TensorFlow session.
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)
summary_writer = tf.summary.FileWriter(summary_path)
saver = tf.train.Saver()

# Optionally load the model.
if load_model == True:
    print('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_path)
    saver.restore(sess,ckpt.model_checkpoint_path)

# Create variables which will be used throughout training.
total_steps = 0
global_buffer = np.zeros([0, 5])
total_reward = []
total_length = []
total_entropy = []
v_losses = []
p_losses = []

# Start training loop
brain_info = env.reset()[default_brain]
state = brain_info.vector_observations[0]

for i in range(total_episodes):
    running_reward = 0
    episode_steps = 0
    ep_history = []
    done = False
    while not done:
        episode_steps += 1
        total_steps += 1
        # Probabilistically pick an action given our network outputs.
        action, entropy, value = sess.run([myAgent.sampled_action, myAgent.entropy], 
                          feed_dict={myAgent.state_in:[state]})
        
        # Take action in environment.
        brain_info = env.step(action)[default_brain]
        
        # Collect resulting reward and new observation.
        state_1 = brain_info.vector_observations[0]
        reward = brain_info.rewards[0]
        done = brain_info.local_done[0]
        ep_history.append([state, action, reward, value, entropy])
        running_reward += reward
        state = state_1
        
        if done and train_model:
            # Add experience trajectory to global buffer.
            ep_history = np.array(ep_history)
            ep_history[:,2] = discount_rewards(ep_history[:,2], y)
            global_buffer = np.concatenate([global_buffer, ep_history])
            
            if i % update_frequency == 0 and i != 0:
                # Update the network using global buffer.
                feed_dict={myAgent.reward_holder:global_buffer[:,2],
                        myAgent.action_holder:global_buffer[:,1],
                        myAgent.state_in:np.vstack(global_buffer[:,0])}
                v_loss, p_loss, _ = sess.run([myAgent.value_loss, 
                                                  myAgent.policy_loss, 
                                                  myAgent.update_model], 
                                                 feed_dict=feed_dict)
                v_losses.append(v_loss)
                p_losses.append(p_loss)
                global_buffer = np.zeros([0, 5])

    total_reward.append(running_reward)
    total_length.append(episode_steps)
    total_entropy.append(np.mean(ep_history[:, 4]))
    total_value.append(np.mean(ep_history[:, 3]))

    # Update our running tally of scores and save information to Tensorboard.
    if i % summary_freq == 0 and i != 0:
        summary = tf.Summary()
        summary.value.add(tag='Info/Mean Reward', simple_value=float(np.mean(total_reward[-summary_freq:])))
        summary.value.add(tag='Info/Value Estimate', simple_value=float(np.mean(total_value[-summary_freq:])))
        summary.value.add(tag='Info/Episode Length', simple_value=float(np.mean(total_length[-summary_freq:])))
        summary.value.add(tag='Info/Entropy', simple_value=float(np.mean(total_entropy[-summary_freq:])))
        summary.value.add(tag='Info/Value Loss', simple_value=float(np.mean(v_losses[-summary_freq:])))
        summary.value.add(tag='Info/Policy Loss', simple_value=float(np.mean(p_losses[-summary_freq:])))
        summary_writer.add_summary(summary, i)
        summary_writer.flush()
        print("Step: {}, Episode: {}, Mean Reward: {}".format(str(total_steps), str(i),
                                                              str(round(np.mean(total_reward[-summary_freq:]), 3))))        
    # Save agent's model
    if i % 1000 == 0 and i != 0:
        saver.save(sess, model_path+'/model-'+str(i)+'.cptk')
        print("Saved Model")
env.close()

In [ ]:
env.close()